![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Trainining a Text Classification Model

In [0]:
 !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv
 
 dbutils.fs.cp("file:/databricks/driver/petfinder-mini.csv", "dbfs:/") 

Out[1]: True

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 spark://10.139.64.10:7077 
 AppName 
 Databricks Shell

## Load dataset

In [0]:
dataframe = pd.read_csv('petfinder-mini.csv')

In [0]:
# In the original dataset "4" indicates the pet was not adopted.

dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [0]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [0]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of ca...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartment. It was shaking so I had to bring it home to p...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresponsible owner at the roadside near some shops in S...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience waiting for her good master, plz call or sms for m...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption. He is the most playful pal we've seen in our pupp...,3,1


In [0]:
dataframe.columns

Out[8]: Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'target'],
      dtype='object')

In [0]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11537 non-null  object
 1   Age           11537 non-null  int64 
 2   Breed1        11537 non-null  object
 3   Gender        11537 non-null  object
 4   Color1        11537 non-null  object
 5   Color2        11537 non-null  object
 6   MaturitySize  11537 non-null  object
 7   FurLength     11537 non-null  object
 8   Vaccinated    11537 non-null  object
 9   Sterilized    11537 non-null  object
 10  Health        11537 non-null  object
 11  Fee           11537 non-null  int64 
 12  Description   11528 non-null  object
 13  PhotoAmt      11537 non-null  int64 
 14  target        11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [0]:
dataframe.target.value_counts()

Out[10]: 1    8457
0    3080
Name: target, dtype: int64

In [0]:
dataframe.Description = dataframe.Description.fillna('- no description -')

## Featurize with Sklearn Column Transformer

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(handle_unknown='ignore', categories='auto'), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [0]:
y.nunique()


Out[13]: 2

In [0]:
X.shape

Out[14]: (11537, 302)

In [0]:
input_dim = X.shape[1]

In [0]:
input_dim

Out[16]: 302

In [0]:
import scipy.sparse

df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df = df.fillna(0)

df['target']= y

df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.171825,0.0,0.137765,0.0,0.000000,0.283609,0.0,0.220992,0.151594,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.144907,0.000000,0.0,0.158869,0.0,0.0,0.000000,0.170202,0.000000,0.192465,0.190783,0.000000,0.132427,0.176639,0.151246,0.000000,0.0,0.000000,0.000000,0.257843,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.228484,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.256152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250843,0.000000,0.0,0.000000,0.000000,0.213817,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.141649,0.000000,0.0,0.000000,0.0,0.172449,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.307225,0.0,0.178193,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.210999,0.209155,0.209619,0.000000,0.000000,0.414527,0.179294,0.0,0.000000,0.000000,0.141336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.257886,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.198538,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.126763,0.113263,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.081638,0.000000,0.105362,0.128186,0.0,0.000000,0.0,0.000000,0.107062,0.086649,0.0,0.000000,0.0,0.0,0.000000,0.125751,0.218471,0.142200,0.000000,0.000000,0.195683,0.000000,0.111746,0.120832,0.0,0.120993,0.000000,0.000000,0.223069,0.117846,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Train with Spark NLP Generic Classifier

**Building a pipeline**

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations. The operation of the classifier is controled by the following methods:

*setEpochsNumber(int)* - Determines how many epochs the model is trained.

*setBatchSize(int)* - Sets the batch size during training.

*setLearningRate(float)* - Sets the learning rate.

*setValidationSplit(float)* - Sets the proportion of examples in the training set used for validation.

*setModelFile(string)* - Loads a model from the specified location and uses it instead of the default model.

*setFixImbalance(boolean)* - If set to true, it tries to balance the training set by weighting the classes according to the inverse of the examples they have.

*setFeatureScaling(string)* - Normalizes the feature factors using the specified method ("zscore", "minmax" or empty for no normalization).

*setOutputLogsPath(string)* - Sets the path to a folder where logs of training progress will be saved. No logs are generated if no path is specified.

In [0]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|col_293|col_294|col_295|col_296|col_297|col_298|            col_299|             col_300|            col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|-0.4524794726808656|  0.9502875792756131|-0.8297616989552165|     1|
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844|     1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows



In [0]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9169
Test Dataset Count: 2368


Training with a custom graph

In [0]:
#or just use the one we already have in the repo
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /databricks/driver/gc_graph
'''

Out[20]: '\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /databricks/driver/gc_graph\n'

In [0]:
%fs mkdirs file:/dbfs/gc_graph

res0: Boolean = true

In [0]:
%fs mkdirs file:/dbfs/generic_logs

res1: Boolean = true

Graph and log folder has been created. <br/>
We will create a custom graph and train the model. <br/>

In [0]:
#!pip install -q tensorflow==2.12.0
#!pip install -q tensorflow-addons

In [0]:
DL_params = {"input_dim": input_dim, 
             "output_dim": y.nunique(), 
             "hidden_layers": [300, 200, 100], 
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


medical.tf_graph.build("generic_classifier",
               build_params=DL_params, 
               model_location="file:/dbfs/gc_graph",
               model_filename="auto")

Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to file:/dbfs/gc_graph/gcl.302.2.pb


In [0]:
#from sparknlp_jsl.base import *


features_asm = medical.FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")

gen_clf = medical.GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setModelFile('file:/dbfs/gc_graph/gcl.302.2.pb')\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("dbfs:/generic_logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm, 
    gen_clf])

In [0]:
clf_model = clf_Pipeline.fit(training_data)

Getting predictions

In [0]:
pred_df = clf_model.transform(test_data)

In [0]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     0|   [1]|
|     1|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [0]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [0]|
|     1|   [1]|
+------+------+
only showing top 20 rows



In [0]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['target'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['target']))

              precision    recall  f1-score   support

           0     0.4406    0.5483    0.4886       507
           1     0.8682    0.8103    0.8382      1861

    accuracy                         0.7542      2368
   macro avg     0.6544    0.6793    0.6634      2368
weighted avg     0.7766    0.7542    0.7634      2368

0.754222972972973


## Create a custom DL architecture with TF

In [0]:
graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("target")\
    .setHiddenLayers([300, 200, 100])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph.pb")

In [0]:
'''
# Create custom graph

# If this method is used, graph folder should be added to training
# model as `.setGraphFolder(graph_folder_path)`

!mkdir gc_graph

medical.tf_graph.print_model_params("generic_classifier")

DL_params = {"input_dim": input_dim,
             "output_dim": y.nunique(),
             "hidden_layers": [300, 200, 100],
             "hidden_act": "tanh",
             'hidden_act_l2':1,
             'batch_norm':1}


medical.tf_graph.build("generic_classifier",
               build_params=DL_params,
               model_location="/content/gc_graph",
               model_filename="auto")
'''

'''
# or just use the one we already have in the repo

!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /content/gc_graph
'''

Out[30]: '\n# or just use the one we already have in the repo\n\n!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/generic_classifier_graph/pet.in1202D.out2.pb -P /content/gc_graph\n'

In [0]:
!mkdir logs

features_asm = medical.FeaturesAssembler()\
    .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
    .setOutputCol("features")

gen_clf = medical.GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.002)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


mkdir: cannot create directory ‘logs’: File exists


In [0]:
%%time

# train 50 epochs (takes around 1 min)

clf_model = clf_Pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph.pb
Build params: {'input_dim': 302, 'output_dim': 2, 'hidden_layers': [300, 200, 100], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph.pb
CPU times: user 1.28 s, sys: 129 ms, total: 1.41 s
Wall time: 35.4 s


In [0]:
import os
log_file_name = os.listdir("logs")[0]

with open("logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

23/11/21 08:00:17 INFO DriverDaemon$: {"metric":"serviceUptime","tags":{"driverPublicDns":"20.185.238.11","numPerClusterInitScriptsV2S3":"0","numPerClusterInitScriptsV2Gcs":"0","opType":"TimerTask","clusterScalingType":"fixed_size","attribute_tag_dust_execution_env":"","driverContainerId":"524ecaa1e16940e184ce401394ca5bea","numPerClusterInitScriptsV2Volumes":"0","clusterState":"Pending","ignoreTerminationEventInAlerting":"false","clusterLogDeliveryEnabled":"false","sparkEnvVarContainsSingleQuotes":"false","driverNodeType":"Standard_DS4_v2","shardName":"","azureSubscriptionId":"bc5674c1-2f09-4eff-8497-b97f5466158f","databricksCommit":"24e017641399d00ca97844e026495137927d24f9","numPerClusterInitScriptsV2Abfss":"0","sparkMasterUrlType":"None","userProvidedRemoteVolumeCount":"0","clusterUnityCatalogMode":"CUSTOM","hostName":"0725-092232-stftp6wi-10-139-64-10","enableJdbcAutoStart":"true","driverInstanceId":"5f6ab79b8c424237be6bea99b4a8bb88","orgId":"7956323724731612","sparkVersion":"12.2.x

In [0]:
pred_df = clf_model.transform(test_data)

In [0]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     0|   [0]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
+------+------+
only showing top 20 rows



In [0]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [0]:
preds_df.sample(10)

,target,result
78,0,0
624,1,1
2057,1,1
1665,1,1
1893,1,1
543,1,1
2056,1,1
657,0,0
1470,1,0
151,1,1


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['target'], preds_df['result'], digits=4))

print (accuracy_score(preds_df['target'], preds_df['result']))


              precision    recall  f1-score   support

           0     0.5467    0.5008    0.5227       631
           1     0.8240    0.8492    0.8364      1737

    accuracy                         0.7563      2368
   macro avg     0.6854    0.6750    0.6796      2368
weighted avg     0.7501    0.7563    0.7528      2368

0.7563344594594594


## get prediction for random input

In [0]:
pd.DataFrame([dataframe.loc[5191].to_dict()])

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Dog,2,Mixed Breed,Female,Black,Brown,Medium,Medium,Yes,Yes,Healthy,10,Friendly and pampered female puppy looking for a beautiful home,2,1


In [0]:
input_X = column_trans.transform(pd.DataFrame([dataframe.loc[0].to_dict()]).drop(['target'], axis=1))

input_y = dataframe.target[0]


In [0]:
input_df = pd.DataFrame.sparse.from_spmatrix(input_X)

input_df.columns = ['col_{}'.format(i) for i in range(input_X.shape[1])]

input_df['target']= input_y

input_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_253,col_254,col_255,col_256,col_257,col_258,col_259,col_260,col_261,col_262,col_263,col_264,col_265,col_266,col_267,col_268,col_269,col_270,col_271,col_272,col_273,col_274,col_275,col_276,col_277,col_278,col_279,col_280,col_281,col_282,col_283,col_284,col_285,col_286,col_287,col_288,col_289,col_290,col_291,col_292,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.171825,0.0,0.137765,0.0,0.0,0.283609,0.0,0.220992,0.151594,0.0,0.0,0.0,0.0,0.0,0.0,0.144907,0.0,0.0,0.158869,0.0,0.0,0.0,0.170202,0.0,0.192465,0.190783,0.0,0.132427,0.176639,0.151246,0.0,0.0,0.0,0.0,0.257843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.452479,0.950288,-0.829762,1


In [0]:
input_spark_df = spark.createDataFrame(input_df)
input_spark_df.show(2)

+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [0]:
clf_model.transform(input_spark_df).select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
+------+------+



# Case Study: Alexa Review Classification

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/amazon_alexa.tsv -P /dbfs/

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/amazon_alexa.tsv', sep='\t')
df

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wr...",1
3,5,31-Jul-18,Charcoal Fabric,"I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the light...",1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in between!!",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, checking time, looking up weather. There are many more ...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my entire home, TV, all my lights, my thermostat, my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound quality isn't great. I mostly use it for commands though...,1


In [0]:
df.verified_reviews = df.verified_reviews.str.lower()

In [0]:
df.feedback.value_counts()

Out[47]: 1    2893
0     257
Name: feedback, dtype: int64

In [0]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (TfidfVectorizer(max_features=1000,  norm='l2', ngram_range=(1, 3)), 'verified_reviews'))

X = column_trans.fit_transform(df.drop(['feedback'], axis=1))

y = df.feedback

In [0]:
import scipy.sparse

sdf = pd.DataFrame.sparse.from_spmatrix(X)

sdf.columns = ['col_{}'.format(i) for i in range(X.shape[1])]

sdf = sdf.fillna(0)

sdf['feedback']= y

sdf.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,col_40,col_41,col_42,col_43,col_44,col_45,col_46,col_47,col_48,col_49,...,col_951,col_952,col_953,col_954,col_955,col_956,col_957,col_958,col_959,col_960,col_961,col_962,col_963,col_964,col_965,col_966,col_967,col_968,col_969,col_970,col_971,col_972,col_973,col_974,col_975,col_976,col_977,col_978,col_979,col_980,col_981,col_982,col_983,col_984,col_985,col_986,col_987,col_988,col_989,col_990,col_991,col_992,col_993,col_994,col_995,col_996,col_997,col_998,col_999,feedback
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.145963,0.146283,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.327234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.324472,0.0,0.149662,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.16371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104466,0.0,0.0,0.0,0.0,0.0,0.210616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [0]:
input_spark_df = spark.createDataFrame(sdf)

In [0]:
(training_data, test_data) = input_spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 2475
Test Dataset Count: 675


In [0]:
#from sparknlp_jsl.base import *

features_asm = medical.FeaturesAssembler()\
      .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
      .setOutputCol("features")
        
gen_clf = medical.GenericClassifierApproach()\
    .setLabelColumn("feedback")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")\
    .setEpochsNumber(50)\
    .setBatchSize(100)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.001)\
    .setOutputLogsPath("file:/databricks/driver/generic_logs")\
   #.setModelFile("/databricks/driver/gc_graph/pet_in1202D_out2.pb")
    

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm, 
    gen_clf])

clf_model = clf_Pipeline.fit(training_data)


In [0]:
pred_df = clf_model.transform(test_data)

preds_df = pred_df.select('feedback','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['result'], preds_df['feedback'], digits=4))

print (accuracy_score(preds_df['result'], preds_df['feedback']))


              precision    recall  f1-score   support

           0     0.5741    0.4627    0.5124        67
           1     0.9420    0.9622    0.9520       608

    accuracy                         0.9126       675
   macro avg     0.7581    0.7124    0.7322       675
weighted avg     0.9055    0.9126    0.9084       675

0.9125925925925926
